<a href="https://colab.research.google.com/github/valedonnet/Logistic-Regression-Project-Tutorial/blob/main/Logistic_Regression_Project_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
!pip install tableone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.2 MB/s eta 0:00:00


In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv",
               sep=';')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [4]:
def is_binary(df_, nums):
    df = df_.copy()
    variables = []
    for var in nums:
        flag = True
        unique = df_[var].unique()
        for value in unique:
            if value not in [0, 1, np.nan, 0.0, 1.0]:
                flag = False
        if flag == True:
            variables.append(var)
    return variables

def breakdown_vars(df, off_binary=False):
    """
    This function allow us categorize accodign to numerical or not
    """
    binaries = is_binary(df, df.columns)
    categorial = []
    nonormal = []
    normal = []
    for t in df.columns:
        if off_binary == False:
          if (df[t].dtypes.name=="object" or df[t].dtypes.name=='category') and  t not in binaries:
            categorial.append(t)
        else:
           if (df[t].dtypes.name=="object" or df[t].dtypes.name=='category'):
            categorial.append(t)
        if (df[t].dtypes=="int64" or df[t].dtypes=="float64") and t not in binaries:
                n,p = stats.shapiro(df[t])
                if p<0.05:
                    nonormal.append(t)
                else:
                    normal.append(t)
    if off_binary == False:
      return categorial, binaries, nonormal, normal
    else:
      return categorial, nonormal, normal

In [5]:
cat, nonormal ,  normal = breakdown_vars(df,  off_binary=True)

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 41188.
  res = hypotest_fun_out(*samples, **kwds)


In [6]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [7]:
#The groupby parameter is required to perform statistical tests and calculate p-values when generating a table with TableOne.

In [8]:
from tableone import TableOne

In [9]:
mytable=TableOne(df, categorical=cat, nonnormal=nonormal, groupby='y',pval=True)

In [10]:
mytable

Grouped by y                                                                                
                                                        Missing                 Overall                      no                     yes P-Value
n                                                                                 41188                   36548                    4640        
age, median [Q1,Q3]                                           0        38.0 [32.0,47.0]        38.0 [32.0,47.0]        37.0 [31.0,50.0]   0.016
job, n (%)                     admin.                                      10422 (25.3)             9070 (24.8)             1352 (29.1)  <0.001
                               blue-collar                                  9254 (22.5)             8616 (23.6)              638 (13.8)        
                               entrepreneur                                  1456 (3.5)              1332 (3.6)               124 (2.7)        
                               housemaid                                     1060 (2.6)               954 (2.6)               106 (2.3)        
                               management                                    2924 (7.1)              2596 (7.1)               328 (7.1)        
                               retired                                       1720 (4.2)              1286 (3.5)               434 (9.4)        
                               self-employed                                 1421 (3.5)              1272 (3.5)               149 (3.2)        
                               services                                      3969 (9.6)             3646 (10.0)               323 (7.0)        
                               student                                        875 (2.1)               600 (1.6)               275 (5.9)        
                               technician                                   6743 (16.4)             6013 (16.5)              730 (15.7)        
                               unemployed                                    1014 (2.5)               870 (2.4)               144 (3.1)        
                               unknown                                        330 (0.8)               293 (0.8)                37 (0.8)        
marital, n (%)                 divorced                                     4612 (11.2)             4136 (11.3)              476 (10.3)  <0.001
                               married                                     24928 (60.5)            22396 (61.3)             2532 (54.6)        
                               single                                      11568 (28.1)             9948 (27.2)             1620 (34.9)        
                               unknown                                         80 (0.2)                68 (0.2)                12 (0.3)        
education, n (%)               basic.4y                                     4176 (10.1)             3748 (10.3)               428 (9.2)  <0.001
                               basic.6y                                      2292 (5.6)              2104 (5.8)               188 (4.1)        
                               basic.9y                                     6045 (14.7)             5572 (15.2)              473 (10.2)        
                               high.school                                  9515 (23.1)             8484 (23.2)             1031 (22.2)        
                               illiterate                                      18 (0.0)                14 (0.0)                 4 (0.1)        
                               professional.course                          5243 (12.7)             4648 (12.7)              595 (12.8)        
                               university.degree                           12168 (29.5)            10498 (28.7)             1670 (36.0)        
                               unknown                                       1731 (4.2)              1480 (4.0)               251 (5.4)        
default, n (%)     

In [11]:
#Existe diferencia significativa en las variables para el evento de interés (y) y el grupo de control.

In [12]:
# Existe multicolinelidad

#SIEMPRE NORMALIZO DESPUES DE SPLIT TRAIN AND TEST
def normalize(array):
  return (array - array.mean())/array.std()

In [13]:
num = normal + nonormal
for numerical in num:
  df[numerical] = normalize(df[numerical].to_numpy())

In [14]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [ ]:
mytable = TableOne(df,categorical=cat, nonnormal=nonormal, groupby='y', pval=True)

In [ ]:
mytable

In [ ]:
mytable.to_excel('Tableone.xlsx')

In [ ]:
cat

In [ ]:
df.columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Crear el objeto OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', drop='first')

# Aplicar OneHotEncoder a las columnas categóricas
ohe_df = pd.DataFrame(ohe.fit_transform(df[['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']]).toarray())

# Actualizar el índice del DataFrame codificado para que coincida con el DataFrame original
ohe_df.index = df.index

# Eliminar las columnas categóricas originales
df.drop(columns=['job', 'marital', 'education', 'default', 'housing', 'loan',
 'contact', 'month', 'day_of_week', 'poutcome'], inplace=True)

# Asignar nombres a las nuevas columnas codificadas
ohe_df.columns = ohe.get_feature_names_out(['job', 'marital', 'education', 'default',
                                            'housing', 'loan', 'contact', 'month',
                                            'day_of_week', 'poutcome'])

# Unir el DataFrame original con el DataFrame codificado
df = df.join(ohe_df)



In [ ]:
df.head()

In [ ]:
df['y'].unique()

In [ ]:
df.info()


In [ ]:
df['y'] = df['y'].map({'yes': 1, 'no': 0})

In [ ]:
corrMatrix = df.corr(method='spearman')
mask = np.zeros_like(corrMatrix)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
     f, ax = plt.subplots(figsize=(40, 30))
     ax = sns.heatmap(corrMatrix, mask=mask, square=True, annot=True, cmap="gray")

In [ ]:
y = df['y']

In [ ]:
X = df[df.columns[df.columns!='y']]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# With
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle = True,
                                                    random_state = 123,
                                                    stratify = y)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [ ]:
clf=LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
preds=clf.predict(X_test)
print(classification_report(y_test,preds))

El modelo funciona muy bien en la clase mayoritaria (clase 0), pero tiene dificultades significativas con la clase minoritaria (clase 1), como se ve en el menor recuerdo y la puntuación F1 para la clase 1.
Esto indica un problema de desequilibrio en el que el modelo puede estar sesgado hacia la clase 0.

In [ ]:
#Quería chequear que era el mismo valor accuracy f1_score del reporte
grid_accuracy = accuracy_score(y_test, preds)
grid_accuracy

In [ ]:
# Hay 10965 muestras de la clase 0: Precisión 93% (de los casos 0 previsto el 93% fueron correctos), Recall 97% (El módelo identifico correctamente el 97% de las instancias verdaderas de calse 0),
#Hay 1392 muestras de clase 1: El 66% de las instancias predichas como clase 1 eran en realidad clase 1. El modelo solo identificó el 43% de las instancias verdaderas de clase 1.


In [ ]:
clf.get_params()

In [ ]:
from sklearn.model_selection import GridSearchCV

# We define a dictionary with the parameters that we want to adjust by hand
hyperparams = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "penalty": ["l1", "l2", "elasticnet", None],
    "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
}

# We initialize the grid. Scoring para indicar con que métrica lo voy a evaluar, puede ser más de uno
#cv:Determina la estrategia de división de la validación cruzada.Si no indico algún int para cv por defecto utiliza la validación cruzada
#quíntuple predeterminada, que divide los datos en cinco pliegues.

grid = GridSearchCV(clf, hyperparams, scoring = "accuracy", cv = 5)
grid

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#Ajustamos los datos definidos en grid para los 140 modelos

grid.fit(X_train, y_train)

print(f"Best hyperparameters: {grid.best_params_}")

In [ ]:
#keys() gives the names of all the metrics and parameters recorded in the cross-validation results.

sorted(grid.cv_results_.keys())

In [ ]:
grid.cv_results_['params']

In [ ]:
print('rank_test_score')
print(grid.cv_results_['rank_test_score'])
print('mean_test_score')
print(grid.cv_results_['mean_test_score'])
print('best_score')
print(grid.best_score_)
print('best_params')
print(grid.best_params_)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
model_grid = LogisticRegression(penalty = "l1", C = 10, solver = "liblinear")
model_grid.fit(X_train, y_train)
y_pred = model_grid.predict(X_test)

grid_accuracy = accuracy_score(y_test, y_pred)
grid_accuracy

In [ ]:
#Puedo extraer del GridSearch el mejor modelo en vez de volver a generarlo en model_grid
best_model = grid.best_estimator_
print(best_model.predict(X_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# We define the parameters we want to adjust
hyperparams = {
    "C": np.logspace(-4, 4, 20),
    "penalty": ["l1", "l2", "elasticnet", None],
    "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
}

# We initialize the random search
random_search = RandomizedSearchCV(clf, hyperparams, n_iter = 100, scoring = "accuracy", cv = 5, random_state = 42)
random_search

In [ ]:
random_search.fit(X_train, y_train)

print(f"Best hyperparameters: {random_search.best_params_}")

In [ ]:
model_random_search = LogisticRegression(penalty = "l1", C = 1.623776739188721, solver = "liblinear")
model_random_search.fit(X_train, y_train)
y_pred = model_random_search.predict(X_test)

random_search_accuracy = accuracy_score(y_test, y_pred)
random_search_accuracy

In [ ]:
#POSIBLES MEJORAS DEL MODELO:
#UNDERSAMPLING
#ELIMINAR VARIABLES "confounding."